In [1]:
# imports
import pandas as pd

In [2]:
# function takes dataframe and transforms into shares
def transform_share(df):
    columns = df.columns
    total = df.sum(axis=1)
    for i in columns:
        if i in ["Municipality", "Years", "Geographic Group"]:
            pass
        else:
            df[i] = df[i]/total
    
    return df

In [3]:
# transform education columns
def transform_edu(df, sex):
    df = df.rename(columns={'With no educational qualifications':'no_' + sex,
                          'Compulsory education 1st cycle':'first_' + sex,
                           'Compulsory education 2nd cycle': "second_" + sex,
                           'Compulsory education 3rd cycle': "third_" + sex,
                           'Upper-secondary': "upper_"+ sex,
                           'Middle level': "middle_"+ sex,
                           'Higher education': "higher_"+ sex
                          })
    return df

In [4]:
# transform type columns
def transform_type(df, sex):
    df = df.rename(columns={'Looking for 1st job':'first_job_' + sex,
                          'Looking for a new job':'new_job_' + sex,
                          })
    return df

In [5]:
# change name to Municipality
def transform_to_muni(df):
    df = df.rename(columns={'Years':'Municipality',
                           "Geographic Group":'Municipality'})
    df.set_index("Municipality", inplace=True)
    return df

In [6]:
# load all files into pandas df's
unemp = pd.read_excel("data_files/2011/unemp_rate_2011.xlsx")
all_age = pd.read_excel("data_files/2011/unemployment_age_2011.xlsx")
female_edu = pd.read_excel("data_files/2011/unemp_female_edu_2011.xlsx")
male_edu = pd.read_excel("data_files/2011/unemp_male_edu_2011.xlsx")
female_type = pd.read_excel("data_files/2011/unemp_female_type_2011.xlsx")
male_type = pd.read_excel("data_files/2011/unemp_male_type_2011.xlsx")

In [7]:
# prep unemp
unemp = transform_to_muni(unemp)
unemp = unemp.rename(columns={'Total':'Unemployment_Rate'})

In [8]:
# prep all_age
all_age = all_age.drop(["Total"], axis=1)
all_age = transform_to_muni(all_age)
all_age = transform_share(all_age)

In [9]:
# prep edu
female_edu = transform_to_muni(female_edu)
female_edu = transform_edu(female_edu, "female")
female_edu = transform_share(female_edu)

male_edu = male_edu.drop(["Geographic Group"], axis=1)
male_edu = transform_edu(male_edu, "male")
male_edu = transform_to_muni(male_edu)
male_edu = transform_share(male_edu)

In [10]:
# prep type
female_type = transform_to_muni(female_type)
female_type = transform_type(female_type, "female")
female_type = transform_share(female_type)

male_type = transform_to_muni(male_type)
male_type = transform_type(male_type, "male")
male_type = transform_share(male_type)

In [11]:
# get a list of all dfs
dfs = [all_age, female_edu, male_edu, female_type, male_type]

In [12]:
# concat all df's by looping over it
for d in dfs:
    unemp = unemp.join(d, on="Municipality")

In [13]:
unemp.head()

,Unemployment_Rate,15 - 24,25 - 34,35 - 44,45 - 54,55 - 64,65 or more,no_female,first_female,second_female,...,first_male,second_male,third_male,upper_male,middle_male,higher_male,first_job_female,new_job_female,first_job_male,new_job_male
Municipality,,,,,,,,,,,,,,,,,,,,,
Arcos de Valdevez,10.2,0.194757,0.242197,0.228464,0.239700,0.094881,0.000000,0.010101,0.148990,0.106061,...,0.306173,0.167901,0.227160,0.162963,0.009877,0.106173,0.272727,0.727273,0.153086,0.846914
Caminha,13.1,0.194503,0.263214,0.205074,0.214588,0.121564,0.001057,0.008097,0.089069,0.101215,...,0.174779,0.152655,0.300885,0.234513,0.013274,0.112832,0.194332,0.805668,0.188053,0.811947
Melgaço,9.7,0.233216,0.307420,0.236749,0.183746,0.038869,0.000000,0.000000,0.095238,0.068027,...,0.198529,0.139706,0.264706,0.242647,0.051471,0.102941,0.292517,0.707483,0.286765,0.713235
Monção,9.8,0.201125,0.260197,0.229255,0.216596,0.092827,0.000000,0.018405,0.076687,0.098160,...,0.257143,0.166234,0.205195,0.244156,0.025974,0.083117,0.300613,0.699387,0.166234,0.833766
Paredes de Coura,11.1,0.211538,0.254808,0.245192,0.199519,0.088942,0.000000,0.000000,0.130208,0.098958,...,0.241071,0.205357,0.258929,0.196429,0.035714,0.062500,0.239583,0.760417,0.214286,0.785714


In [14]:
unemp.to_pickle("./unemp.pkl")